# Embedded Methods
## Adaptive-Runge–Kutta


I show here how the Runge–Kutta-Fehlberg method works. I'll implement a general
RKF solver, in which one can give the parameters of *any* stage and order method
and run it. Once I finish understanding it, I'll implement RKF45, and then more!

I found that [thesis](http://www2.imm.dtu.dk/pubdb/views/edoc_download.php/6607/pdf/imm6607.pdf)
to be very useful.


The explicit Runge–Kutta (RK) method can be made, efficiently, adaptive. This is done
by the so-called embedded methods, which basically involves the calculation of each step
by two different solvers, and adapting the step-size until their difference is below some acceptable
value. The proceedure is similar to the classical RM methods (one of order $p$ and the other $p+1$).

Lets assume that we have a differential equation
$$
\dfrac{d\vec{y}}{dt}=f(\vec{y},t) \;,
$$
with  given $\vec{y}(0)$ (as always $t \in [0,1]$). We follow the iteration:[$^1$](#1)

$$
\vec{y}_{n+1}=\vec{y}_{n}+ \sum_{i=1}^{s} b_i \vec{k}_i \\
\vec{y}^{\star}_{n+1}=\vec{y}_{n}+ \sum_{i=1}^{s} b_i^{\star} \vec{k}_i \;,
$$
with
$$
\vec{k}_{i}=f\Bigg(\vec{y}_{n}+h \Big(\sum_{j=1}^{i-1}a_{ji}\vec{k}_{i} \Big), t_{n}+h c_{i}   \Bigg)\;.
$$

Having the two estimates for the next step, we can estimate the error 

$$
\epsilon \equiv \left|\vec{y}_{n+1}- \vec{y}^{\star}_{n+1} \right|=
\left| \sum_{i} (b_{i}-b_{i}^{\star}) \vec{k}_{i} \right|\, h \;.
$$

Then, if $\epsilon \geq \epsilon_0$ ($\epsilon_0$ is some desirable error), we adjust the stepsize as[$^2$](#footnote2)

$$ 
h \to \beta h \left (\dfrac{\epsilon_0}{\epsilon} \right)^{1/p} \;,
$$

and calculate again $\vec{y}_{n+1}$ and $\vec{y}_{n+1}^{\star}$ until  $\epsilon < \epsilon_0$.
When $\epsilon < \epsilon_0$, we accept $y_{n+1}$ ($y_{n+1}^{\star}$ is just for stepsize control!),
we increase the stepsize a bit

$$ 
h \to \beta h \left (\dfrac{\epsilon_0}{\epsilon} \right)^{1/(p+1)} \;,
$$

and move to the next step.


$$\begin{array}{}
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
\hline
\end{array}$$ 
<span id="footnote1">$^1$ </span>Butcher tableau in this case is written as

$$
\begin{array}{c|cccc}
0      & 0      &   0   &      0& \dots & 0& 0\\
c_2    & a_{21} &   0   &   0   & \dots & 0& 0\\
c_3    & a_{31} & a_{32}&      0& \dots & 0& 0\\
\vdots & \vdots & \vdots& \vdots&\ddots &\ddots& \vdots\\
c_s    & a_{s1} & a_{s2}& a_{s3}& \dots & a_{s s-1}& 0 \\
\hline
 p      & b_1    & b_2  & b_3 & \dots & b_{s-1} & b_s \\
 p+1      & b_1^{\star}    & b_2^{\star}  & b_3^{\star} & \dots & b_{s-1}^{\star} & b_s^{\star}
\end{array}
$$



<span id="footnote2">$^2$ </span>
Note that $\beta \approx 1$ is an input parameter used to adjust
the aggresiveness of the changes in h.


### RKF Algorithm
In an algorithmic way, the RKF method can be written as:

```bash
Inputs-> system_eqs=dydt, initial_condition=y0, No_equations=Neqs, initial_step_size=h0, 
            minimum_step_size=hmin, maximum_step_size=hmax, maximum_No_steps=Nsteps, 
            relative_tolerance=eps_rel, absolute_tolerance=eps_abs, beta=b, method=RKpq
#Initializations
tn=0

#allocate lists we will need: 
Define k[Neqs][s], ak[Neqs], bk[Neqs], bstark[Neqs], ynext[Neqs], ynext_star[Neqs], yn[Neqs],

#Allow steps and solution to be arrays with maximum number of steps (or use dynamic arrays or something)
Define steps[Nsteps], solution[Neqs][Nsteps] 
        


for eq in [1,2,...,Neq]
    yn[eq]=y0[eq]
    ynext[eq]=0
    ynext_star[eq]=0
    delta[eq]=0

h=h0

while tn<1 and step<Nsteps do
    
    h_stop=True#change to False when you find a suitable h
    while h_stop do
        #calculate \vec{k}
        for stage in [1,2...,s] do
            for eq in [1,2...,Neqs] do
                ak[eq]=sum(a[j,i]*k[i], i in [1,...,stage-1] )
            done

            for eq in [1,2,...,Neq] do
                self.k[eq][stage]=dydt( sum(yn[i] +_ak[i], i in [1,2,...,Neqs]) ,tn+ c[stage]*h)[eq]
            done
        done

        #calculate \vec{y}_{n+1} and \vec{y}_{n+1}_{\star}
        ## first calculate sum b*k         
        for eq in [1,2,...,Neqs]do
            bk[eq]=sum(b[i]*k[eq][i]*h, i in [1,2,...,s])
            bstark[eq]=sum(bstar[i]*k[eq][i]*h, i in [1,2,...,s])
        done


        for eq in [1,2,...,Neqs]do
            ynext[eq]=yn[eq]+ bk[eq]
            ynext_star[eq]=yn[eq]+ bstark[eq]
            delta[eq]=ynext[eq]-ynext_star[eq]
            
        done
        
        #find current error and relative error 
        abs_err_n=abs(delta) #you may take the euclidean abs, or something else.
        
        #calculate relative error
        for eq in [1,2,...,Neqs]do
            rel_delta[eq]=(ynext[eq]-ynext_star[eq])/ynext[eq]
        done
        rel_err_n=abs(rel_delta)
        
        
        #adjust h. There are more refined ways to do this, but this is a start.  
        ##################################################################################################
        if (err_n>eps_abs or rel_err_n> eps_rel) then
            h=beta*h0*(eps_abs/abs_err)^(1/p) #p is given by the method
        else 
            h=beta*h0*(eps_abs/abs_err)^(1/(p+1))
            h_stop=False
        fi
        
        #if you have limits apply them
        if h>hmax then    
            h=hmax
        fi
        
        if h<hmin then
            h=hmin
        fi
        ##################################################################################################
    done
    
    
    for eq in [1,2,...,Neqs] do
        yn[eq]=ynext[eq]
        solution[eq][step]=yn[eq]
    done
    
    h0=h
    
    
    tn=tn+h
    if tn>1 then
        tn=1
    fi
    
    steps[step]=tn
    
done

return steps, solution

#note that for error monitoring, it may be useful to return an
#array with the errors and stepsize at each step.
```



For example the RKF45


todo...

In [1]:
import numpy as np

import matplotlib
#matplotlib.use('WebAgg')
#matplotlib.use('Qt4Cairo')
#matplotlib.use('Qt5Cairo')
matplotlib.use('nbAgg')
import matplotlib.pyplot as plt

plt.rcParams['font.family']='serif'
plt.rcParams['font.size']=10
plt.rcParams['mathtext.fontset']='stixsans'

In [2]:
#define RKF45 parameters
class RKF45:
    def __init__(self):
        self.s=
        self.p=
        self.c=
        self.b=
        self.bstar=
        self.a=[ [0 for j in range(self.s)] for i in range(self.s)]
        
        
rkf45=RKF45()

In [10]:

class RKF:
    
    '''
       This is still exRK, but I will adjust it to RKF
    
    '''
    
    def __init__(self,n_eqs,diffeq,init_cond,N,RK_method):
        self.step_number=N
        self.step_size=1./(N-1)#constant step size.
        self.dydt=diffeq.dydt#the differential equation to be integrated
        self.number_of_eqs=n_eqs#number of equations
        
        self.steps=[0 for i in range(N)]#list of steps, ie t
        
        #initiate list of solutions for every t, ie y^(i)_{n}
        self.solution=[0 for i in range(self.number_of_eqs)]
        for eq_i in range(self.number_of_eqs):
            self.solution[eq_i]=[0 for i in range(N)]
            
        for eq_i in range(self.number_of_eqs):
            self.solution[eq_i][0]=init_cond[eq_i]#the first step is the initial condition
        
        self.current_step=0#initiate a counter (when this becomes equal to N, the solver terminates)
        
        self.end=False# to be changed to True when self.current_step=N
        
        #get the parameters that define the method
        self.s=RK_method.s
        self.a=RK_method.a
        self.b=RK_method.b
        self.c=RK_method.c
        
        
        self.yn=[0 for i in range(self.number_of_eqs)]#this is initiated to hold current steps
        
        self.k=[0 for i in range(self.number_of_eqs)]#this is initiated to hold all ks
        for eq_i in range(self.number_of_eqs):
            self.k[eq_i]=[0 for i in range(self.s)]
        
        
    # function to calculate the \sum_{i=1}^{current stage-1} a_{current stage,i} \vec{k}_{i}*step_size
    def sum_ak(self,stage):
        ak=[0 for i in range(self.number_of_eqs)]
        
        for eq_i in range(self.number_of_eqs):
            for i in range(stage-1):
                ak[eq_i]+=self.a[stage][i]*self.k[eq_i][i]*self.step_size
        
        return ak
    
    # function to calculate the \sum_{i=1}^{s} b_{i} \vec{k}_{i}*step_size
    def sum_bk(self):
        bk=[0 for i in range(self.number_of_eqs)]
        for eq_i in range(self.number_of_eqs):
            for i in range(self.s):
                bk[eq_i]+=self.b[i]*self.k[eq_i][i]*self.step_size
        return bk

    def next_step(self):
        '''
        Get the next step.
        '''
        if self.current_step>=self.step_number-1:
            self.end=True 
        else:
            self.current_step+=1
            tn=self.current_step*self.step_size
            self.steps[self.current_step]=tn
            
            #define a list which holds previous point (makes the code slower, but more transparent) 
            for eq_i in range(self.number_of_eqs):
                self.yn[eq_i]=self.solution[eq_i][self.current_step-1]
    
            #this is \vec{k}_1.  
            for eq_i in range(self.number_of_eqs):
                self.k[eq_i][0]=self.dydt(self.yn,tn)[eq_i]
            
            #once you have \vec{k}_1, find the others.
            for stage in range(1,self.s):
                #since \sum_{i=1}^{s} a_{stage,i} \vec{k}_{i}*step_size is the same for all 
                #equations in a given stage, call here self.sum_ak
                _ak=self.sum_ak(stage)
                
                #get a d\vec{y}/dt needed for \vec_{k}_{stage}
                _dydt=self.dydt(self.yn+_ak,tn+self.c[stage]*self.step_size )
                for eq_i in range(self.number_of_eqs):
                    self.k[eq_i][stage]=_dydt[eq_i]
                    
            
            #calculate \sum_{i=1}^{s} b_{i} \vec{k}_{i}*step_size
            _bk=self.sum_bk()
            for eq_i in range(self.number_of_eqs):
                #calculate \vec{y}_{n+1}=\vec{y}_{n}+\sum_{i=1}^{s} b_{i} \vec{k}_{i}*step_size
                self.solution[eq_i][self.current_step]=self.yn[eq_i]+_bk[eq_i]
                
            
                
            
    def solve(self):
        '''
        Run  next_step until self.end becomes True.
        '''
        while not self.end:
            self.next_step()